# Aggregation Pipeline Demo

This notebook demonstrates the spatial and temporal aggregation pipeline.  
It walks through each step: building the fine network, optionally constructing a coarse network, 
computing node features, generating distance matrices, applying clustering, and producing 
aggregated datasets.  

**Purpose**: Show how to run the full pipeline as well as inspect each component step-by-step.  
**Inputs**: Raw data under `/DATA/raw/` (downloaded from Google Drive, see [README.md](../README.md) for details.) 
**Outputs**: Aggregated datasets and cached metrics under `/DATA/processed/`  


In [1]:
import sys
import pathlib

PROJECT_ROOT = pathlib.Path().cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) Quick start: run the full pipeline once

This cell shows the *shortest possible* way to run the aggregation pipeline end-to-end.

You choose a granularity (`"fine"` → ~385 nodes, or `"coarse"` → 17 zones derived from the fine network), set a year, and provide simple weights and hyperparameters. The call:

- builds the selected network,
- computes features,
- performs spatial aggregation (to `n_representative_nodes`),
- performs temporal aggregation (to `k_representative_days`), and
- returns results plus a **version hash** (which is used to create versioned folders on disk in `results/joint_aggregation_results/v<hash>/`).

In [2]:
from src.aggregation.pipeline import StaticPreprocessor, DynamicProcessor

# --- Step 1. Static preprocessing ---
# Choose "fine" (385 nodes) or "coarse" (17 zones)
preproc = StaticPreprocessor(granularity="coarse", year=2013).preprocess()

# --- Step 2. Dynamic run with hyperparameters ---
dyn = DynamicProcessor(preprocessor=preproc)

weights={
        'position': 1.0,
        'time_series': 0.8,
        'duration_curves': 1.2,
        'ramp_duration_curves': 1.0,
        'intra_correlation': 1.0,
        'inter_correlation': 1.0
    }

results, version_hash, day_weights, metadata = dyn.run_with_hyperparameters(
    weights=weights,
    n_representative_nodes=10,
    k_representative_days=12
)

print("Results saved under version:", version_hash)
print("Representative day weights:", day_weights)

print("Version hash:", version_hash)
print("Available result blocks:", list(results.keys()))  # ['spatiotemporal', 'temporal_only', 'original', 'clusters']


c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:140: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.y,
c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:141: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.x


Cached metrics loaded from C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\distance_metrics\v202586c2.
Results saved to C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\joint_aggregation_results\v4b02ae39
Results saved under version: 4b02ae39
Representative day weights: {11: 11, 37: 14, 40: 22, 47: 48, 48: 68, 53: 9, 185: 37, 199: 26, 202: 25, 218: 42, 232: 43, 248: 20}
Version hash: 4b02ae39
Available result blocks: ['spatiotemporal', 'temporal_only', 'original', 'clusters']


## 2) Component details
### Configure the aggregation run

We first instantiate a `Config` that controls data year, network granularity (choose `"fine"` ~385 nodes or `"coarse"` 17 zones), and which features to compute.  
Running `config.help()` prints a short overview of all knobs and how they affect the pipeline.  
The same `Config` object will be reused when building networks, computing features, distances, and saving results. 

The full structure of `Config` (both preprocessing and hyperparameters) is detailed in [src/aggregation/README.md](../src/aggregation/README.md). 


In [4]:
from src.aggregation.settings import Config

config = Config(
    year=2013,
    granularity="coarse",
    active_features=['position', 'time_series', 'duration_curves', 'ramp_duration_curves', 'intra_correlation']
)

# Display configuration help.
config.help()

Configuration Overview:

1. Data Preproc (Immutable):
   Contains data processing parameters. Key attributes:
   - year (int): The data year (2007-2013).
   - granularity (str): Data granularity; allowed: 'coarse', 'fine'.
   - active_features (list): Features to include; allowed: 'position', 'time_series', 'duration_curves',
      'ramp_duration_curves', 'intra_correlation'.
2. Model Hyper (Mutable):
   Contains model configuration parameters. Key attributes:
   - n_representative_nodes (int): Number of representative nodes.
   - k_representative_days (int): Number of representative days (1–365).
   - inter_correlation (bool): Whether inter-node correlation is included.
   - kmed_seed (int): Seed for KMedoids (0 means no seed).
   - kmed_n_init (int): Number of KMedoids initializations.
   - weights (dict): Weights for various features. Can be manually set or auto-generated based on 
      data_preproc.active_features and model_hyper.inter_correlation

3. Path (Immutable):
   Contains

### Building the fine network

This cell shows how to explicitly build the **fine network** (≈385 nodes) and inspect the raw time-series shapes:

- `FineNetwork(config)` loads and cleans **wind/solar** capacity factors and **demand**.
- `.build_fine_ntw()` returns a dict with:
    - `'nodes'`: a DataFrame with `Lat`, `Lon`
    - `'time_series'`: a dict with three DataFrames (`'wind'`, `'solar'`, `'demand'`), each of shape **T × N** (rows = hours, columns = nodes).

We also show how to access **population & county** utilities (via `utils`) indirectly through the network builder.

In [6]:
from src.aggregation.utils import FineNetwork

cfg = Config(
    year=2013,
    granularity="fine",
    active_features=["position", "time_series", "duration_curves", "ramp_duration_curves", "intra_correlation"],
)

fine_builder = FineNetwork(cfg)
fine_data = fine_builder.build_fine_ntw()

print("Nodes columns:", fine_data["nodes"].columns.tolist())
for k, df in fine_data["time_series"].items():
    print(f"{k}: shape {df.shape}")  # (hours, nodes)


c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:140: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.y,
c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:141: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.x


Nodes columns: ['Lat', 'Lon']
wind: shape (8760, 385)
solar: shape (8760, 385)
demand: shape (8760, 385)


### Choosing coarse network (17 zones) vs staying fine

To build a **coarse network (17 zones)**, we:

- build the fine network first,
- snap each fine node to the nearest coarse zone center (provided by `coarse_node_file` in `settings.PathConfig`), and
- **medoid** the wind/solar time series per zone (demand is **summed**).

The coarse network has the same structure as the fine one, but with **17** columns.

In [7]:
from src.aggregation.utils import CoarseNetwork

cfg_coarse = Config(
    year=2013,
    granularity="coarse",
    active_features=["position", "time_series", "duration_curves", "ramp_duration_curves", "intra_correlation"],
)

fine_builder = FineNetwork(cfg_coarse)
fine_data = fine_builder.build_fine_ntw()

coarse_builder = CoarseNetwork(cfg_coarse, fine_data)
coarse_data = coarse_builder.build_coarse_ntw()

print("Coarse nodes:", coarse_data["nodes"].shape[0])
for k, df in coarse_data["time_series"].items():
    print(f"{k}: shape {df.shape}")  # time series now have 17 columns


c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:140: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.y,
c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:141: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.x


Coarse nodes: 17
wind: shape (8760, 17)
solar: shape (8760, 17)
demand: shape (8760, 17)


### Building the `Network` object and inspecting features

`Network` bundles **nodes + time series** and computes **features** for each node based on `active_features`:

- `'position'`: `(Lat, Lon)` tuple per node
- `'time_series'`: dict `{ 'wind': 1D array, 'solar': 1D array, 'demand': 1D array }` per node
- `'duration_curves'`: per-series sorted values (normalized [0,1])
- `'ramp_duration_curves'`: per-series sorted absolute hour-to-hour ramps (normalized [0,1])
- `'intra_correlation'`: per-node pairwise correlations between series types within the node (intra)

You can also subset dates via `start_day`/`end_day` (e.g., `"06-01"` to `"08-31"`).

In [10]:
from src.aggregation.utils import Network

ntw = Network(
    nodes_df=coarse_data["nodes"],
    time_series=coarse_data["time_series"],
    config=cfg_coarse,                 # uses active_features from Config
    start_day=None, end_day=None
)

# Inspect features for the first node (index 0)
f0 = ntw.features[0]
print("Feature keys at node 0:", list(f0.keys()))
print("Position:", f0["position"])
print("Time-series keys:", list(f0["time_series"].keys()))
print("Duration-curves keys:", list(f0["duration_curves"].keys()))
print("Ramp-duration-curves keys:", list(f0["ramp_duration_curves"].keys()))
print("Intra-correlation keys:", list(f0["intra_correlation"].keys()))


Feature keys at node 0: ['position', 'time_series', 'duration_curves', 'ramp_duration_curves', 'intra_correlation']
Position: (42.642711, -70.865107)
Time-series keys: ['wind', 'solar', 'demand']
Duration-curves keys: ['wind', 'solar', 'demand']
Ramp-duration-curves keys: ['wind', 'solar', 'demand']
Intra-correlation keys: [('wind', 'solar'), ('wind', 'demand'), ('solar', 'demand')]


### Distance metrics and caching

`SpatialAggregation.DistanceCalculator.compute_metrics(...)` builds a **distance matrix per feature** (e.g., `'position'`, `'time_series'`, `'intra_correlation'`), then **normalizes** each to [0,1].

The `SpatialAggregation` object **caches** these metrics on disk:

- Folder: `results/distance_metrics/v<hash>/`
- Files:
    - `metrics.npz` — NumPy archive with one array per metric key
    - `metadata.json` — configuration (immutable preproc & `inter_correlation`) used to compute the hash

When you request `aggregation.distance_metrics` again with the same config, it loads from cache.

In [11]:
from src.aggregation.models import SpatialAggregation

# Build aggregation object from features
agg = SpatialAggregation(node_features=ntw.features, config=cfg_coarse)

# First access → compute & save
dm = agg.distance_metrics
print("Cached feature keys:", list(dm.features.keys()))

# Inspect the cache folder
cache_root = cfg.path.distance_metrics
print("Cache root:", cache_root)
# The exact versioned folder:
versioned = agg._io.get_metrics_path()
print("Versioned cache folder:", versioned)


Cached metrics loaded from C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\distance_metrics\v202586c2.
Cached feature keys: ['position', 'time_series', 'ramp_duration_curves', 'duration_curves', 'intra_correlation', 'inter_correlation']
Cache root: C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\distance_metrics
Versioned cache folder: C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\distance_metrics\v202586c2


### Spatial aggregation (k-medoids or MIP)

We combine the normalized distance metrics using **weights** from `Config.model_hyper.weights`.  

Two methods are available:

- **k-medoids** (`Clusterer`) — a fast heuristic that runs multiple random initializations.  
  - `Config.model_hyper.n_init`: number of random restarts (higher = more robust, slower).  
  - `Config.model_hyper.seed`: RNG seed for reproducibility (42 for my thesis).  

- **Optimization** (`Optimizer` with Gurobi) — solves a facility-location–like MILP to select medoids exactly.  

**Shared hyperparameters for both methods:**  
- `Config.model_hyper.weights`: vector of feature weights for distance computation.  
- `Config.model_hyper.n_representative`: number of medoids (clusters) to select.  

**Outputs (same for both methods):**  
- `clusters`: dictionary mapping medoid → list of assigned members  
  (e.g. `{5: [5, 17, 22], 11: [11, 2, 9], ...}`)  
- `representatives`: list of medoid indices (e.g. `[5, 11, 34, ...]`)

The pipeline runs this automatically, but here we show a **manual** call.


In [14]:
# Manually combine metrics and run k-medoids
weights = cfg_coarse.model_hyper.weights  # auto-generated from active_features (+ inter_correlation)
total = SpatialAggregation._combine_metrics(dm, weights)

# Option A: k-medoids
kmed = SpatialAggregation.Clusterer(cfg_coarse)
k = cfg_coarse.model_hyper.n_representative_nodes
res_km = kmed.cluster(total, k=k)
print("K-medoids representatives:", res_km["representatives"])

# Option B: exact optimization
opt = SpatialAggregation.Optimizer(cfg_coarse)
res_opt = opt.solve(total)
print("Optimization representatives:", res_opt["representatives"])


K-medoids representatives: [1, 2, 7, 11, 16]
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 307 rows, 306 columns and 884 nonzeros
Model fingerprint: 0xd68da009
Variable types: 0 continuous, 306 integer (306 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 307 rows, 306 columns, 884 nonzeros
Variable types: 0 continuous, 306 integer (306 binary)
Found heuristic solution: objective 6.0508429

Root relaxation: objective 5.098466e+00, 90 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

### Temporal aggregation (representative days)

Given spatial clusters, `TemporalAggregation`:

- samples/normalizes the time series of clustered nodes,
- builds a **day-by-feature matrix**, and
- clusters into **`k_representative_days`** using k-medoids (distance: Euclidean between day vectors).

It returns clusters of days and their representatives.

In [16]:
from src.aggregation.models import TemporalAggregation

spatial_results = res_km  # or res_opt
temp = TemporalAggregation(cfg_coarse, ntw.features, spatial_results)
temporal_results = temp.aggregate()

print("Representative days:", sorted(temporal_results["representatives"]))
print("#Clusters (days):", len(temporal_results["clusters"]))


Representative days: [2, 47, 48, 53, 177, 185, 199, 211, 218, 236]
#Clusters (days): 10


### Saving complete aggregation results

`Results(...)` packages **three blocks** and saves them to disk under a versioned folder based on both **preproc** and **hyperparams**:

- Folder: `results/joint_aggregation_results/v<hash>/`
- Saved:
    - `spatiotemporal/` — aggregated nodes/branches + rep-day time series (CSVs)
    - `temporal_only/` — original nodes/branches + rep-day time series (CSVs)
    - `original/` — original nodes/branches + full time series (CSVs)
    - `clustering/` — JSONs for spatial & temporal clusters
    - `metadata.json` — config (preproc + hyperparameters, including weights)

In [17]:
from src.aggregation.utils import Results

results_obj = Results(
    config=cfg,
    data=fine_data,                     # original (fine) network data
    spatial_agg_results=spatial_results,
    temporal_agg_results=temporal_results,
    auto_save=True
)

print("Saved blocks:", list(results_obj.results.keys()))
print("Base path:", cfg.path.joint_aggregation_results)


Results saved to C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\joint_aggregation_results\vc3e25595
Saved blocks: ['spatiotemporal', 'temporal_only', 'original', 'clusters']
Base path: C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\joint_aggregation_results
